In [ ]:
%load_ext autoreload
%autoreload 2

In [26]:
import os
import shutil
from yolo_tools import mseg_attribute_update_gt, data_merge, data_check, mseg2seg, ref_split

from yolo_mask_crop import myolo_crop, myolo_crop_mp
from classify_tools import ref_split_class
from yolo_sta import yolo_sta
from select_defect_data import obj_record, select_by_predict
from yolo_result_analysis import model_pred_compare


In [ ]:
defect_list = ['deformation', 'broken', 'abandonment', 'corrosion']
level_list = ['no', 'medium', 'high']

ps_mseg_c6_dir = r'/localnvme/data/billboard/ps_data/psdata735_mseg_c6_check0624'
bd_mseg_c6_dir = r'/localnvme/data/billboard/bd_data/data626_mseg_c6_check0624'
fuse_mseg_c6_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_check0624'
fuse_seg_c6_dir = r'/localnvme/data/billboard/fused_data/data1361_seg_c6_check0624'

In [ ]:
mseg_attribute_update_gt(
    os.path.join(ps_mseg_c6_dir, 'labels'),
    os.path.join(ps_mseg_c6_dir, 'labels'),
)

In [ ]:
attribute_path = os.path.join(bd_mseg_c6_dir, 'attribute.yaml')
check_result_ps1 = data_check(ps_mseg_c6_dir, mdet=True, attribute_path=attribute_path)
check_result_bd1 = data_check(bd_mseg_c6_dir, mdet=True, attribute_path=attribute_path)
check_result_ps2 = data_check(ps_mseg_c6_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')
check_result_bd2 = data_check(bd_mseg_c6_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')

if len(check_result_ps1+check_result_bd1 + check_result_ps2+check_result_bd2) == 0:
    data_merge(ps_mseg_c6_dir,
               bd_mseg_c6_dir,
               fuse_mseg_c6_dir)
    shutil.copy2(
        os.path.join(bd_mseg_c6_dir, 'class.txt'),
        os.path.join(fuse_mseg_c6_dir, 'class.txt'),
    )
    shutil.copy2(
        os.path.join(bd_mseg_c6_dir, 'attribute.yaml'),
        os.path.join(fuse_mseg_c6_dir, 'attribute.yaml'),
    )
else:
    print(f'find {check_result_ps1} and {check_result_bd1}')
    print(f'find {check_result_ps2} and {check_result_bd2}')

In [ ]:
dataset_dir = fuse_mseg_c6_dir
image_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')
class_file = os.path.join(dataset_dir, 'class.txt')
attribute_file = os.path.join(dataset_dir, 'attribute.yaml')

image_crop_dir = os.path.join(dataset_dir, 'images_crop_box')
sta_dir=os.path.join(dataset_dir, "labels_sta")
obj2img_path=os.path.join(dataset_dir, "images_crop.json")
sta_att_path=os.path.join(sta_dir, "sta_attribute.csv")
box_path=os.path.join(sta_dir, "sta_box.csv")
info_path=os.path.join(sta_dir, "info.csv")

# myolo_crop(image_dir, labels_dir, image_crop_dir, class_file,
#            attribute_file=attribute_file, seg=True, annotation=True,
#            save_method='attribute',
#            crop_method='without_background_box_shape')

myolo_crop_mp(image_dir, labels_dir, image_crop_dir, class_file,
           attribute_file=attribute_file, seg=True, annotation=True,
           save_method='attribute',
           crop_method='without_background_box_shape')

In [ ]:
yolo_sta(
    img_dir=None,
    gt_dir=labels_dir,
    result_dir=sta_dir,
    class_path=class_file,
    attribute_path=attribute_path,
    seg=True,
)

In [ ]:
obj_record(sta_att_path, box_path, obj2img_path, info_path)

In [ ]:
dataset_dir = fuse_mseg_c6_dir
sta_dir=os.path.join(dataset_dir, "labels_sta")
info_path=os.path.join(sta_dir, "info.csv")
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
val_path = os.path.join(dataset_dir, 'val.txt')

ref_split_class(ref_path=val_path,
                info_path=info_path,
                input_dir=image_crop_dir,
                defect_list = defect_list,
                level_list = level_list)


In [ ]:
data_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6'
labels_dir = os.path.join(data_dir, 'labels')
images_dir = os.path.join(data_dir, 'images')
cls_val_dir = os.path.join(data_dir, 'cls_val207')
val_dir = os.path.join(r'/localnvme/project/ultralytics/runs/msegment', 'val207')
predict_dir = os.path.join(val_dir, 'labels')
labels_npy_dir = os.path.join(val_dir, 'labels_npy')
predict_npy_dir = os.path.join(val_dir, 'predicts_npy')
compare_dir = predict_dir+'val207_compare'

In [ ]:
model_pred_compare(labels_dir, predict_dir, save_dir=compare_dir, seg=True, with_conf=False, with_att=True, threshold=0.5, att_num=4, image_width=1, image_height=1, labels_npy_dir=labels_npy_dir, predict_npy_dir=predict_npy_dir)

In [ ]:
select_by_predict(compare_dir, images_dir, cls_val_dir,
                  defect_list = defect_list,
                  level_list = level_list)

In [ ]:
data_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_check0618'
labels_dir = os.path.join(data_dir, 'labels')
images_dir = os.path.join(data_dir, 'images')
cls_val_dir = os.path.join(data_dir, 'cls_val208')
val_dir = os.path.join(r'/localnvme/project/ultralytics/runs/msegment', 'val208')
predict_dir = os.path.join(val_dir, 'labels')
labels_npy_dir = os.path.join(val_dir, 'labels_npy')
predict_npy_dir = os.path.join(val_dir, 'predicts_npy')
compare_dir = predict_dir+'val208_compare'

In [ ]:
model_pred_compare(labels_dir, predict_dir, save_dir=compare_dir, seg=True, with_conf=False, with_att=True, threshold=0.5, att_num=4, image_width=1, image_height=1, labels_npy_dir=labels_npy_dir, predict_npy_dir=predict_npy_dir)

In [ ]:
select_by_predict(compare_dir, images_dir, cls_val_dir,
                  defect_list = defect_list,
                  level_list = level_list)

In [ ]:
mseg2seg(fuse_mseg_c6_dir, fuse_seg_c6_dir)

In [27]:
ref_split(
    ref_path = os.path.join(fuse_mseg_c6_dir, 'val.txt'),
    img_dir = os.path.join(fuse_seg_c6_dir, 'images'),
    label_dir = os.path.join(fuse_seg_c6_dir, 'labels'),
    add_suffix='',
)

img check: 100%|██████████| 1361/1361 [00:00<00:00, 78984.51it/s]

1223 save to /localnvme/data/billboard/fused_data/data1361_seg_c6_check0624/train.txt,
138 save to /localnvme/data/billboard/fused_data/data1361_seg_c6_check0624/val.txt!
